## 高級中等學校分析
[高級中等學校校別資料(含進修部(學校))](https://data.gov.tw/dataset/6238)；[高級中等學校學生休學人數](https://data.gov.tw/dataset/46318)；[高級中等學校學生學籍喪失人數](https://data.gov.tw/dataset/46317)

1. 本分析以109學年度台灣高等中學校別資料為起點，檢視學校數量、教師、學生數等數據
2. 衍伸分析相較於前次台灣景氣高峰2014/10時(以103學年度為代表)，高等中學數據相對表現
3. 衍伸分析休學及學級喪失人數

In [1]:
import pandas as pd
base109 = pd.read_csv('109_high_base0.csv',thousands=",")
#print(base109.head())
#base109.astype({'學校代碼':'string'})
base109.set_index('學校代碼')
base109['teach_fulltime'] = base109['專任教師數男'] + base109['專任教師數女']
base109['teach_parttime'] = base109['兼任教師數男'] + base109['兼任教師數女']
base109['emp'] = base109['職員數男'] + base109['職員數女']
base109['stud'] = base109['學生數男'] + base109['學生數女']
base109['grad'] = base109['上學年畢業生數男'] + base109['上學年畢業生數女']
print(base109.tail())
print(base109.info())

###嘗試紀錄：因為學生數男及學生數女欄位有千分位數，被理解為object
###嘗試了許多方法如下，但都失敗，最終發現可以在讀取時直接指定 thousands="," 才解決問題
#base109.loc[:,'學生數男'] = base109['學生數男'].str.replace('*,*','')
#base109['學生數女'].replace(',','')
#base109['學生數男'].unique()
#base109.astype({'學校代碼':'string','學生數男':'string','學生數女':'string'})
#base109.loc[:,'學生數男'] = base109['學生數男'].replace('*,*','')
#base109['學生數男'].unique()
#,'學生數男':'int64','學生數女':'int64'})

     縣市代碼 縣市名稱    學校代碼        學校名稱  專任教師數男  專任教師數女  兼任教師數男  兼任教師數女  職員數男  \
523    15  花蓮縣  151G91  私立正德高中進修學校       0       0      23       5     0   
524     3  桃園市  03C301    誠正中學桃園分校       0       0       0       0     0   
525     4  新竹縣  04C301        誠正中學       0       0       0       0     0   
526     7  彰化縣  07C301    誠正中學彰化分校       0       0       0       0     0   
527    12  高雄市  12C301        明陽中學       0       0       0       0     0   

     職員數女  班級數  學生數男  學生數女  上學年畢業生數男  上學年畢業生數女  teach_fulltime  \
523     0    3    71     0        25         0               0   
524     0   11   177     0         3         0               0   
525     0   13   210     0        19         0               0   
526     0   14   148    58         0         0               0   
527     0    6    61     1         0         0               0   

     teach_parttime  emp  stud  grad  
523              28    0    71    25  
524               0    0   177     3  
525               0    0   21

In [2]:
base109['teach_fulltime'] = base109['專任教師數男'] + base109['專任教師數女']
base109.head()

,縣市代碼,縣市名稱,學校代碼,學校名稱,專任教師數男,專任教師數女,兼任教師數男,兼任教師數女,職員數男,職員數女,班級數,學生數男,學生數女,上學年畢業生數男,上學年畢業生數女,teach_fulltime,teach_parttime,emp,stud,grad
0,30,臺北市,400144,國立臺灣戲曲學院附設高職部,31,40,0,0,0,0,21,134,275,47,88,71,0,0,409,135
1,14,臺東縣,140222,國立臺東專科學校附設高職部,34,37,0,0,8,3,36,549,252,124,58,71,0,11,801,182
2,1,新北市,010301,國立華僑高級中等學校,50,84,5,4,6,19,50,659,851,181,261,134,9,25,1510,442
3,1,新北市,011301,私立淡江高中,36,68,0,0,6,13,29,486,534,181,169,104,0,19,1020,350
4,1,新北市,011302,私立康橋高中,24,24,0,0,4,1,18,415,389,127,104,48,0,5,804,231


### 前置作業 - 進行加工，把學校代碼依照意義拆出來
[學校代碼定義](https://zh.wikipedia.org/zh-tw/%E5%AD%B8%E6%A0%A1%E4%BB%A3%E7%A2%BC)：
學校代碼由2位學校所在地代號、1位設立別、1位學校級別和2位流水號組成，其中大專院校及軍警學校之學校代碼只採用後四碼。

設立別：
0  國立
1  私立
2  省立
3、4 市、縣立 (直轄市、省轄市、縣)
5  鄉、鎮、市立(縣轄市)
M  軍警學校
C  矯正學校

學校級別：
0 大學
1 學院
2 專科
3 高級中學
4 高級職校
5 國民中學
6、7、8 國民小學
A 大專進修學校
B 高中進修學校
C 高職進修學校
D 國中補校
E 國小補校
F 特教學校
G 監獄補校
R 宗教研修學院
H 社區大學
K、W、X、Y、Z 幼稚園
M 臺商子弟學校
S 海外臺灣學校

In [3]:
#做兩張對照表等等可以merge
#為了簡化說明，把各種層級設立的學校增加分類為“公立”，特殊的軍警及矯正分為”其他“
import re
c = [['0','國立','公立'],['1','私立','私立'],['2','省立','公立'],['3','市、縣立 (直轄市、省轄市、縣)','公立'],
     ['4','市、縣立 (直轄市、省轄市、縣)','公立'],['5','鄉、鎮、市立(縣轄市)','公立'],['M','軍警學校','其他'],
     ['C','矯正學校','其他']]
cate_table = pd.DataFrame(c,columns=['cate_code', 'cate_name','cate_name_mj'])

l = [['0','大學'],['1','學院'],['2','專科'],['3','高級中學'],['4','高級職校'],['5','國民中學'],
     ['6','國民小學'],['7','國民小學'],['8','國民小學'],['A','大專進修學校'],['B','高中進修學校'],
     ['C','高職進修學校'],['D','國中補校'],['E','國小補校'],['F','特教學校'],['G','監獄補校'],
     ['R','宗教研修學院'],['H','社區大學'],['K','幼稚園'],['W','幼稚園'],['X','幼稚園'],['Y','幼稚園'],['Z','幼稚園'],
     ['M','臺商子弟學校'],['S','海外臺灣學校']]
lv_table = pd.DataFrame(l,columns=['lv_code', 'lv_name'])

s = [['01','新北市'],['30','臺北市'],['02','宜蘭縣'],['31','松山區'],['03','桃園市'],['32','信義區'],
     ['04','新竹縣'],['33','大安區'],['05','苗栗縣'],['34','中山區'],['07','彰化縣'],['35','中正區'],
     ['08','南投縣'],['36','大同區'],['09','雲林縣'],['37','萬華區'],['10','嘉義縣'],['38','文山區'],
     ['13','屏東縣'],['39','南港區'],['14','臺東縣'],['40','內湖區'],['15','花蓮縣'],['41','士林區'],
     ['16','澎湖縣'],['42','北投區'],['17','基隆市'],['64','高雄市'],['18','新竹市'],['12','高雄市'],
     ['120','嘉義市'],['50','高雄市'],['266','臺中市'],['51','鹽埕區'],['06','臺中市'],['152','鼓山區'],
     ['19','臺中市'],['253','左營區'],['67','臺南市'],['54','楠梓區'],['11','臺南市'],['155','三民區'],
     ['21','臺南市'],['256','新興區'],['71','金門縣'],['58','苓雅區'],['72','連江縣'],['59','前鎮區'],
     ['80','大陸地區'],['60','旗津區'],['90','海外地區'],['61','小港區']]
site_table = pd.DataFrame(s,columns=['site_code', 'site_name'])

#print(cate_table,'\n',lv_table,'\n',reg_table)

In [4]:
base109['site'] = base109['學校代碼'].str[0:2]
base109['cate'] = base109['學校代碼'].str[2]
base109['lv'] = base109['學校代碼'].str[3]
#print(base109.tail())
#把設立別跟學校級別串接進去，之後比較方便分析
base109 = base109.merge(cate_table, how='left', left_on='cate', right_on='cate_code')
base109 = base109.merge(lv_table, how='left', left_on='lv', right_on='lv_code')
#print(base109.tail())
base109.tail()

,縣市代碼,縣市名稱,學校代碼,學校名稱,專任教師數男,專任教師數女,兼任教師數男,兼任教師數女,職員數男,職員數女,...,stud,grad,site,cate,lv,cate_code,cate_name,cate_name_mj,lv_code,lv_name
523,15,花蓮縣,151G91,私立正德高中進修學校,0,0,23,5,0,0,...,71,25,15,1,G,1,私立,私立,G,監獄補校
524,3,桃園市,03C301,誠正中學桃園分校,0,0,0,0,0,0,...,177,3,03,C,3,C,矯正學校,其他,3,高級中學
525,4,新竹縣,04C301,誠正中學,0,0,0,0,0,0,...,210,19,04,C,3,C,矯正學校,其他,3,高級中學
526,7,彰化縣,07C301,誠正中學彰化分校,0,0,0,0,0,0,...,206,0,07,C,3,C,矯正學校,其他,3,高級中學
527,12,高雄市,12C301,明陽中學,0,0,0,0,0,0,...,62,0,12,C,3,C,矯正學校,其他,3,高級中學


### Q1：全台灣有幾間高級中等學校？各個地區分別有幾間？

In [5]:
total = base109.shape[0]
local = base109.groupby(['縣市名稱']).agg({'學校代碼':'count'}).sort_values(by = '學校代碼',ascending=False).reset_index()
#local = local.reset_index()
local = pd.DataFrame(local)
#local
a = '依照各個縣市分別計算數量，由多到少的地區分別是：\n'
#print(local['縣市代碼'])
for i in range(local.shape[0]):
    a = a + local['縣市名稱'][i] + '有' + str(local['學校代碼'][i]) + '間學校 '
print('全台灣總共有',total,'間高級中等學校，',a)

全台灣總共有 528 間高級中等學校， 依照各個縣市分別計算數量，由多到少的地區分別是：
臺北市有69間學校 新北市有64間學校 高雄市有56間學校 臺中市有52間學校 臺南市有49間學校 桃園市有36間學校 彰化縣有27間學校 雲林縣有22間學校 苗栗縣有19間學校 屏東縣有19間學校 南投縣有15間學校 花蓮縣有14間學校 嘉義市有13間學校 新竹市有12間學校 宜蘭縣有12間學校 基隆市有12間學校 新竹縣有11間學校 臺東縣有11間學校 嘉義縣有10間學校 金門縣有2間學校 澎湖縣有2間學校 連江縣有1間學校 


### Q2：有哪些不同種類的高級中等學校？分別有多少？

In [6]:
print(base109['lv_name'].unique())
base109.groupby('lv_name').agg({'學校代碼':'count'}).sort_values(by = '學校代碼',ascending=False)

['學院' '專科' '高級中學' '高級職校' '高中進修學校' '高職進修學校' '特教學校' '監獄補校']


,學校代碼
lv_name,
高級中學,363
高級職校,153
監獄補校,4
特教學校,2
高中進修學校,2
高職進修學校,2
學院,1
專科,1


In [7]:
#想特別檢查一下“學院”跟“專科”這些比較少量出現的資料
base109.loc[base109['lv_name'].isin(['學院','專科'])]
#'監獄補校','特教學校','高中進修學校','高職進修學校',

,縣市代碼,縣市名稱,學校代碼,學校名稱,專任教師數男,專任教師數女,兼任教師數男,兼任教師數女,職員數男,職員數女,...,stud,grad,site,cate,lv,cate_code,cate_name,cate_name_mj,lv_code,lv_name
0,30,臺北市,400144,國立臺灣戲曲學院附設高職部,31,40,0,0,0,0,...,409,135,40,0,1,0,國立,公立,1,學院
1,14,臺東縣,140222,國立臺東專科學校附設高職部,34,37,0,0,8,3,...,801,182,14,0,2,0,國立,公立,2,專科


### Q3：如果依照設立別(ex:公立、私立)來分的話，台灣有多少種高級中等學校？分別有幾間？有多少老師跟學生？

In [8]:
base109_cate = base109.groupby('cate_name').agg({'學校代碼':'count','專任教師數男':'sum','專任教師數女':'sum',
                                                 '兼任教師數男':'sum','兼任教師數女':'sum','學生數男':'sum','學生數女':'sum',
                                                '職員數男':'sum','職員數女':'sum','班級數':'sum',
                                                 '上學年畢業生數男':'sum','上學年畢業生數女':'sum',})

base109_cate['teacher_num'] = base109_cate[['專任教師數男','專任教師數女','兼任教師數男','兼任教師數女']].sum(axis=1)
base109_cate['student_num'] = base109_cate[['學生數男','學生數女']].sum(axis=1)
print('台灣有',base109_cate.shape[0],'種高級中等學校，數量、老師數量、學生數如下表：')
base109_cate[['學校代碼','teacher_num','student_num']]
#.sort_values(by = '學校代碼',ascending=False)

台灣有 4 種高級中等學校，數量、老師數量、學生數如下表：


,學校代碼,teacher_num,student_num
cate_name,,,
國立,139,15524,167949
市、縣立 (直轄市、省轄市、縣),168,23265,207122
矯正學校,4,0,655
私立,217,17738,234019


In [9]:
#已經做好的依照設立別做的表格，可以做後續研究
base109_cate

,學校代碼,專任教師數男,專任教師數女,兼任教師數男,兼任教師數女,學生數男,學生數女,職員數男,職員數女,班級數,上學年畢業生數男,上學年畢業生數女,teacher_num,student_num
cate_name,,,,,,,,,,,,,,
國立,139,6548,7858,454,664,93624,74325,1204,2102,5766,29569,25795,15524,167949
市、縣立 (直轄市、省轄市、縣),168,8467,13810,371,617,112267,94855,1084,2510,6464,33807,31689,23265,207122
矯正學校,4,0,0,0,0,596,59,0,0,44,22,0,0,655
私立,217,6255,8421,1611,1451,122872,111147,1135,2490,6752,41618,39721,17738,234019


### Q4：以師生比來看，私立學校有比較低的師生比嗎？國立跟地方政府設立的學校師生比誰高誰低？

In [10]:
base109_cate_student = (1 / base109_cate.div(base109_cate['student_num'], axis=0)).round(2)
#result_df = df.iloc[:, :-1].div(df['除數'], axis=0)
base109_cate_student

,學校代碼,專任教師數男,專任教師數女,兼任教師數男,兼任教師數女,學生數男,學生數女,職員數男,職員數女,班級數,上學年畢業生數男,上學年畢業生數女,teacher_num,student_num
cate_name,,,,,,,,,,,,,,
國立,1208.27,25.65,21.37,369.93,252.94,1.79,2.26,139.49,79.90,29.13,5.68,6.51,10.82,1.0
市、縣立 (直轄市、省轄市、縣),1232.87,24.46,15.00,558.28,335.69,1.84,2.18,191.07,82.52,32.04,6.13,6.54,8.90,1.0
矯正學校,163.75,inf,inf,inf,inf,1.10,11.10,inf,inf,14.89,29.77,inf,inf,1.0
私立,1078.43,37.41,27.79,145.26,161.28,1.90,2.11,206.18,93.98,34.66,5.62,5.89,13.19,1.0


In [11]:
print('私立學校並沒有比較低的師生比，相反的，它有著最高的師生比， 1 個老師對',base109_cate_student['teacher_num']['私立'],'個學生。')
print('地方政府設立的學校有著最低的師生比， 1 個老師對',base109_cate_student['teacher_num']['市、縣立 (直轄市、省轄市、縣)'],'個學生。')

私立學校並沒有比較低的師生比，相反的，它有著最高的師生比， 1 個老師對 13.19 個學生。
地方政府設立的學校有著最低的師生比， 1 個老師對 8.9 個學生。


### Q5：疫情前(103年)狀況如何？相較於109年有什麼改變？

In [12]:
#匯入103年資料，並且完成初步處理
base103 = pd.read_csv('103_high_base0.csv',thousands=",")
base103.set_index('學校代碼')
base103['teach_fulltime'] = base103['專任教師數男'] + base103['專任教師數女']
base103['teach_parttime'] = base103['兼任教師數男'] + base103['兼任教師數女']
base103['emp'] = base103['職員數男'] + base103['職員數女']
base103['stud'] = base103['學生數男'] + base103['學生數女']
base103['grad'] = base103['上學年畢業生數男'] + base103['上學年畢業生數女']
#print(base103.info())

base103['site'] = base103['學校代碼'].str[0:2]
base103['cate'] = base103['學校代碼'].str[2]
base103['lv'] = base103['學校代碼'].str[3]

#把設立別跟學校級別串接進去，之後比較方便分析
base103 = base103.merge(cate_table, how='left', left_on='cate', right_on='cate_code')
base103 = base103.merge(lv_table, how='left', left_on='lv', right_on='lv_code')

base103.head(3)

,學校代碼,學校名稱,專任教師數男,專任教師數女,兼任教師數男,兼任教師數女,職員數男,職員數女,班級數,學生數男,...,stud,grad,site,cate,lv,cate_code,cate_name,cate_name_mj,lv_code,lv_name
0,400144,國立臺灣戲曲學院附設高職,28,41,119,91,0,0,21,122,...,400,151,40,0,1,0,國立,公立,1,學院
1,140222,國立臺東專科學校附設高職,40,33,21,11,0,0,36,656,...,935,194,14,0,2,0,國立,公立,2,專科
2,010301,國立華僑中學,46,97,4,8,7,22,48,804,...,1696,504,01,0,3,0,國立,公立,3,高級中學


In [13]:
#做出by設立種類的統計表
base103_cate = base103.groupby('cate_name').agg({'學校代碼':'count','專任教師數男':'sum','專任教師數女':'sum',
                                                 '兼任教師數男':'sum','兼任教師數女':'sum','學生數男':'sum','學生數女':'sum',
                                                '職員數男':'sum','職員數女':'sum','班級數':'sum',
                                                 '上學年畢業生數男':'sum','上學年畢業生數女':'sum',})

base103_cate['teacher_num'] = base103_cate[['專任教師數男','專任教師數女','兼任教師數男','兼任教師數女']].sum(axis=1)
base103_cate['student_num'] = base103_cate[['學生數男','學生數女']].sum(axis=1)
base103_cate

,學校代碼,專任教師數男,專任教師數女,兼任教師數男,兼任教師數女,學生數男,學生數女,職員數男,職員數女,班級數,上學年畢業生數男,上學年畢業生數女,teacher_num,student_num
cate_name,,,,,,,,,,,,,,
國立,166,8059,9571,1699,1898,140580,116010,1303,2576,7364,45007,39851,21227,256590
市、縣立 (直轄市、省轄市、縣),138,6951,11843,790,804,97802,85342,699,1901,5054,30003,27719,20388,183144
私立,214,8444,10831,2927,2783,201164,177968,1405,3128,9367,69157,65310,24985,379132


In [14]:
diff_cate_109to103 = base109_cate.sub(base103_cate)
diff_cate_109to103

,學校代碼,專任教師數男,專任教師數女,兼任教師數男,兼任教師數女,學生數男,學生數女,職員數男,職員數女,班級數,上學年畢業生數男,上學年畢業生數女,teacher_num,student_num
cate_name,,,,,,,,,,,,,,
國立,-27.0,-1511.0,-1713.0,-1245.0,-1234.0,-46956.0,-41685.0,-99.0,-474.0,-1598.0,-15438.0,-14056.0,-5703.0,-88641.0
市、縣立 (直轄市、省轄市、縣),30.0,1516.0,1967.0,-419.0,-187.0,14465.0,9513.0,385.0,609.0,1410.0,3804.0,3970.0,2877.0,23978.0
矯正學校,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
私立,3.0,-2189.0,-2410.0,-1316.0,-1332.0,-78292.0,-66821.0,-270.0,-638.0,-2615.0,-27539.0,-25589.0,-7247.0,-145113.0


1. 從103年到109年，國立學校減少，地方政府設立學校增加，懷疑是有國立學校轉為地方管理，也確實查到有關新聞。  
新聞：[桃園9所國立高中職 明年確定改市立](https://news.ltn.com.tw/news/local/paper/1123457)  
2. 學生數量跟老師數量都有明顯減少，以私立學校減少數量最多，少了14.5萬人。
3. 私立學校總數增加了3間，但班級、老師及學生人數都大幅減少，可以進一步分析變化原因

### Q6：從103年到109年，師生比有什麼變化？

In [15]:
#做出師生比表格
base103_cate_student = (1 / base103_cate.div(base103_cate['student_num'], axis=0)).round(2)
base103_cate_student

,學校代碼,專任教師數男,專任教師數女,兼任教師數男,兼任教師數女,學生數男,學生數女,職員數男,職員數女,班級數,上學年畢業生數男,上學年畢業生數女,teacher_num,student_num
cate_name,,,,,,,,,,,,,,
國立,1545.72,31.84,26.81,151.02,135.19,1.83,2.21,196.92,99.61,34.84,5.70,6.44,12.09,1.0
市、縣立 (直轄市、省轄市、縣),1327.13,26.35,15.46,231.83,227.79,1.87,2.15,262.01,96.34,36.24,6.10,6.61,8.98,1.0
私立,1771.64,44.90,35.00,129.53,136.23,1.88,2.13,269.84,121.21,40.48,5.48,5.81,15.17,1.0


In [16]:
for index, row in base103_cate_student.iterrows():
    print('在', index, '類別：\n師生比從103年的',row['teacher_num'],'變為109年的',base109_cate_student['teacher_num'][index])

在 國立 類別：
師生比從103年的 12.09 變為109年的 10.82
在 市、縣立 (直轄市、省轄市、縣) 類別：
師生比從103年的 8.98 變為109年的 8.9
在 私立 類別：
師生比從103年的 15.17 變為109年的 13.19


### Q7：從103年到109年，私立學校消失幾家？增設幾家？私立學校的老師大幅下降，是因為停止營運嗎？

In [40]:
#合併103年資料與109年資料
merge_103and109 = pd.merge(base103,base109,how='outer', left_on='學校代碼', right_on='學校代碼')
#print(merge_103and109.info())
'''merge_103and109 = merge_103and109[['學校代碼',
                                  '學校名稱_x','班級數_x','teach_fulltime_x','teach_parttime_x','emp_x','stud_x','grad_x',
                                  '學校名稱_y','班級數_y','teach_fulltime_y','teach_parttime_y','emp_y','stud_y','grad_y']]'''
merge_103and109 = merge_103and109.reindex(columns=['學校代碼','學校名稱_x','學校名稱_y','班級數_x','班級數_y',
                                                   'teach_fulltime_x','teach_fulltime_y','teach_parttime_x',
                                                   'teach_parttime_y','emp_x','emp_y','stud_x','stud_y',
                                                   'grad_x','grad_y'])

#print(merge_103and109.info())
merge_103and109['site'] = merge_103and109['學校代碼'].str[0:2]
merge_103and109['cate'] = merge_103and109['學校代碼'].str[2]
merge_103and109['lv'] = merge_103and109['學校代碼'].str[3]

#把設立別跟學校級別串接進去，之後比較方便分析
merge_103and109 = merge_103and109.merge(cate_table, how='left', left_on='cate', right_on='cate_code')
merge_103and109 = merge_103and109.merge(lv_table, how='left', left_on='lv', right_on='lv_code')
merge_103and109 = merge_103and109.merge(site_table, how='left', left_on='site', right_on='site_code')

merge_103and109.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 563 entries, 0 to 562
Data columns (total 25 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   學校代碼              563 non-null    object 
 1   學校名稱_x            518 non-null    object 
 2   學校名稱_y            528 non-null    object 
 3   班級數_x             518 non-null    float64
 4   班級數_y             528 non-null    float64
 5   teach_fulltime_x  518 non-null    float64
 6   teach_fulltime_y  528 non-null    float64
 7   teach_parttime_x  518 non-null    float64
 8   teach_parttime_y  528 non-null    float64
 9   emp_x             518 non-null    float64
 10  emp_y             528 non-null    float64
 11  stud_x            518 non-null    float64
 12  stud_y            528 non-null    float64
 13  grad_x            518 non-null    float64
 14  grad_y            528 non-null    float64
 15  site              563 non-null    object 
 16  cate              563 non-null    object 
 1

In [41]:
###練習使用函數
#定義函數
def in_out(x):
    if pd.isnull(x['學校名稱_x']) and pd.notnull(x['teach_fulltime_y']):
        return '1'
    elif pd.notnull(x['學校名稱_x']) and pd.isnull(x['teach_fulltime_y']):
        return '-1'
    else:
        return '0'

#創建新的欄位
merge_103and109['status'] = merge_103and109.apply(in_out, axis=1)
merge_103and109['status'].unique()

array(['0', '-1', '1'], dtype=object)

In [47]:
a = merge_103and109.loc[merge_103and109['cate_name'] == '私立']
#pvt_change = 
temp = a.groupby(['cate_name','status']).agg({'學校代碼':'count',
                                                 '班級數_x':'sum','班級數_y':'sum',
                                                 'teach_fulltime_x':'sum','teach_fulltime_y':'sum',
                                                 'teach_parttime_x':'sum','teach_parttime_y':'sum',
                                                 'emp_x':'sum','emp_y':'sum','stud_x':'sum','stud_y':'sum',
                                                 'grad_x':'sum','grad_y':'sum'}).reset_index()
temp
#,'lv_name'

,cate_name,status,學校代碼,班級數_x,班級數_y,teach_fulltime_x,teach_fulltime_y,teach_parttime_x,teach_parttime_y,emp_x,emp_y,stud_x,stud_y,grad_x,grad_y
0,私立,-1,7,129.0,0.0,216.0,0.0,168.0,0.0,105.0,0.0,3962.0,0.0,2002.0,0.0
1,私立,0,207,9238.0,6682.0,19059.0,14324.0,5542.0,3031.0,4428.0,3484.0,375170.0,232522.0,132465.0,81015.0
2,私立,1,10,0.0,70.0,0.0,352.0,0.0,31.0,0.0,141.0,0.0,1497.0,0.0,324.0


In [56]:
print('私立學校有',temp['學校代碼'][0],'家結束營業，導致消失',
      temp['班級數_x'][0],'個班級與',temp['teach_fulltime_x'][0]+temp['teach_parttime_x'][0],'個老師')
print('持續營業的私立學校有',temp['學校代碼'][1],'家，但縮減了營業規模導致',
      temp['班級數_x'][1]-temp['班級數_y'][1],'個班級消失並減少',
      (temp['teach_fulltime_x'][1]+temp['teach_parttime_x'][1])-
      (temp['teach_fulltime_y'][1]+temp['teach_parttime_y'][1]),
      '個老師')
print('規模縮減並非因為特定學校的倒閉，而有更廣泛的原因')


私立學校有 7 家結束營業，導致消失 129.0 個班級與 384.0 個老師
持續營業的私立學校有 207 家，但縮減了營業規模導致 2556.0 個班級消失並減少 7246.0 個老師
規模縮減並非因為特定學校的倒閉，而有更廣泛的原因


### Q8：從103年到109年，私立學校老師、學生的大幅下降，是發生在哪種類型的學校？

In [68]:
temp = a.groupby(['cate_name','lv_name']).agg({'學校代碼':'count',
                                                 '班級數_x':'sum','班級數_y':'sum',
                                                 'teach_fulltime_x':'sum','teach_fulltime_y':'sum',
                                                 'teach_parttime_x':'sum','teach_parttime_y':'sum',
                                                 'emp_x':'sum','emp_y':'sum','stud_x':'sum','stud_y':'sum',
                                                 'grad_x':'sum','grad_y':'sum'}).reset_index()
temp['teach_x'] = temp['teach_fulltime_x']+temp['teach_parttime_x']
temp['teach_y'] = temp['teach_fulltime_y']+temp['teach_parttime_y']
temp['cls_change'] = temp['班級數_y'] - temp['班級數_x']
temp['teach_change'] = temp['teach_y'] - temp['teach_x']
temp['stud_change'] = temp['stud_y'] - temp['stud_x']
temp
temp = temp.reindex(columns=['cate_name','lv_name','學校代碼','班級數_x',
                             'cls_change','teach_x','teach_change','stud_x','stud_change'])
temp

,cate_name,lv_name,學校代碼,班級數_x,cls_change,teach_x,teach_change,stud_x,stud_change
0,私立,監獄補校,3,9.0,0.0,49.0,-8.0,262.0,-2.0
1,私立,高中進修學校,2,72.0,-6.0,120.0,-82.0,2699.0,-639.0
2,私立,高級中學,155,5708.0,-1360.0,16658.0,-3525.0,236092.0,-78753.0
3,私立,高級職校,63,3558.0,-1244.0,8117.0,-3614.0,139288.0,-65369.0
4,私立,高職進修學校,1,20.0,-5.0,41.0,-18.0,791.0,-350.0


In [73]:
print('私立高級中學減少了最多的班級數，從103年到109年減少了',-1 * temp['cls_change'][2],
      '個班級，相當於減少','{:.2%}'.format(-1 * temp['cls_change'][2]/temp['班級數_x'][2]))
print('私立高級職校減少了最多的老師數，從103年到109年減少了',-1 * temp['teach_change'][3],
      '個老師，相當於減少','{:.2%}'.format(-1 * temp['teach_change'][3]/temp['teach_x'][3]))
print('私立高級中學減少了最多的學生數，從103年到109年減少了',-1 * temp['stud_change'][2],
      '個學生，相當於減少','{:.2%}'.format(-1 * temp['stud_change'][2]/temp['stud_x'][2]))

#'{:.2%}'.format(num)

私立高級中學減少了最多的班級數，從103年到109年減少了 1360.0 個班級，相當於減少 23.83%
私立高級職校減少了最多的老師數，從103年到109年減少了 3614.0 個老師，相當於減少 44.52%
私立高級中學減少了最多的學生數，從103年到109年減少了 78753.0 個學生，相當於減少 33.36%


### Q9：從103年到109年，私立學校老師在哪個地區減少最多？

In [84]:
#                                                   'grad_x','grad_y'])
temp = a.groupby(['site_name']).agg({'學校代碼':'count',
                                                 '班級數_x':'sum','班級數_y':'sum',
                                                 'teach_fulltime_x':'sum','teach_fulltime_y':'sum',
                                                 'teach_parttime_x':'sum','teach_parttime_y':'sum',
                                                 'emp_x':'sum','emp_y':'sum','stud_x':'sum','stud_y':'sum',
                                                 'grad_x':'sum','grad_y':'sum'}).reset_index()
temp['teach_x'] = temp['teach_fulltime_x']+temp['teach_parttime_x']
temp['teach_y'] = temp['teach_fulltime_y']+temp['teach_parttime_y']
temp['cls_change'] = temp['班級數_y'] - temp['班級數_x']
temp['teach_change'] = temp['teach_y'] - temp['teach_x']
temp['stud_change'] = temp['stud_y'] - temp['stud_x']
temp

temp = temp.reindex(columns=['site_name','學校代碼','班級數_x',
                             'cls_change','teach_x','teach_change','stud_x','stud_change']).sort_values(by='teach_change').reset_index()

temp.head()

,index,site_name,學校代碼,班級數_x,cls_change,teach_x,teach_change,stud_x,stud_change
0,16,新北市,32,1272.0,-410.0,3628.0,-959.0,51524.0,-20394.0
1,21,臺中市,24,1460.0,-341.0,4048.0,-957.0,65097.0,-22285.0
2,22,臺南市,23,787.0,-275.0,2293.0,-848.0,30245.0,-13464.0
3,20,桃園市,17,1179.0,-155.0,2595.0,-497.0,51231.0,-14832.0
4,13,屏東縣,7,212.0,-98.0,589.0,-313.0,7439.0,-3680.0


In [85]:
print('新北市的私立高級中等學校減少了最多的老師，從103年到109年減少了',-1 * temp['teach_change'][0],
      '個老師，相當於減少','{:.2%}'.format(-1 * temp['teach_change'][0]/temp['teach_x'][0]))

新北市的私立高級中等學校減少了最多的老師，從103年到109年減少了 959.0 個老師，相當於減少 26.43%


### Q10：從103年到109年，哪個區域私立學校老師減少最少(甚至增加)？

In [87]:
temp = temp.sort_values(by='teach_change',ascending = False).reset_index()
temp.head()

,level_0,index,site_name,學校代碼,班級數_x,cls_change,teach_x,teach_change,stud_x,stud_change
0,29,4,北投區,4,81.0,-13.0,259.0,15.0,2983.0,-807.0
1,28,23,臺東縣,3,50.0,-4.0,104.0,-1.0,1646.0,-577.0
2,27,27,萬華區,1,3.0,-3.0,23.0,-23.0,93.0,-93.0
3,26,12,小港區,1,14.0,-14.0,30.0,-30.0,315.0,-315.0
4,25,11,宜蘭縣,2,32.0,-5.0,158.0,-35.0,1263.0,-182.0


In [89]:
print(temp['site_name'][0],'的私立高級中等學校老師增加最多，從103年到109年增加了',temp['teach_change'][0],
      '個老師，相當於增加','{:.2%}'.format(temp['teach_change'][0]/temp['teach_x'][0]))

北投區 的私立高級中等學校老師增加最多，從103年到109年增加了 15.0 個老師，相當於增加 5.79%


In [ ]:
###另外一種比較差的建立條件判斷的方式
###未完成，留參
#merge_base_103to109.info()
#merge_base_103to109['專任教師數男_y'].isnull()
merge_base_103to109.loc[merge_base_103to109['專任教師數男_y'].isnull()]
merge_base_103to109['in'] =  merge_base_103to109.apply(lambda x: 'new' if pd.isnull(x['學校代碼']) else '', axis=1)
merge_base_103to109['out'] =  merge_base_103to109.apply(lambda x: 'broke' if pd.isnull(x['專任教師數男_y']) else '', axis=1)
merge_base_103to109['in_out'] = merge_base_103to109['in'] + merge_base_103to109['out']
#, axis=1)
#merge_base_103to109['專任教師數男_y'].notnull() and 
#df['new_column'] = df.apply(lambda x: 'yes' if x['col1'] > 2 and x['col2'] < 40 else 'no', axis=1)
merge_base_103to109['in_out'].unique()


In [ ]:
## chatGPT的參考程式碼留存
# 创建一个DataFrame
data = {'Animal': ['Falcon', 'Falcon', 'Parrot', 'Parrot'],
        'Max Speed': [380., 370., 24., 26.],
        'Class': ['Bird', 'Bird', 'Bird', 'Bird']}
df = pd.DataFrame(data)

# 按Animal和Class列分组，计算每个组的平均值、最大值和最小值
grouped = df.groupby(['Animal', 'Class']).agg({'Max Speed': ['mean', 'max', 'min']})

print(grouped)
# 将多层索引转换为单层索引
grouped.columns = ['_'.join(col) for col in grouped.columns.values]

print(grouped)